In [347]:
'''
Apply EM algorithm to protein sequences
'''

'\nApply EM algorithm to protein sequences\n'

In [348]:
import sys
import argparse

import numpy as np

In [349]:
def normalize(array):
    return array / sum(array)

In [350]:
def create_data(input_path, seqlen):
    fin = open(input_path, 'r')
    lines = fin.readlines()
    # exclude first line which is for sequence description
    lines = lines[1:]
    if seqlen <= 0:
        # if seqlen <= 0, use actual sequence length (excluding '\n')
        seqlen = len(lines[0])-1
    array = np.array([list(line.strip()[:seqlen]) for line in lines if line.strip() != ''])
    print("created data with size (%d, %d)" % (len(array), seqlen))
    
    return array

In [351]:
def create_dictionary(data):
    dict = {}
    idx = 0
    for char in np.nditer(data):
        if char.tolist() not in dict:
            dict[char.tolist()] = idx
            idx += 1
    print("created dictionary with %d characters" % len(dict))

    return dict

In [352]:
def create_membership(seqsize, numfamily):
    
    print("created membership matrix of size (%d, %d)" % (seqsize, numfamily))
    return np.apply_along_axis(
        normalize, axis=1, arr=np.random.rand(seqsize, numfamily))

In [353]:
def create_prob(numfamily, numchar, seqlen):
    
    print("created probability matrix of size (%d, %d, %d)" % (numfamily, numchar, seqlen))
    return np.apply_along_axis(
        normalize, axis=1, arr=np.random.rand(numfamily, numchar, seqlen))

In [354]:
def compute_membership(seq, family, dict, prob_mat):
    likelihood = 1.0
    for col, char in enumerate(seq):
        likelihood *= prob_mat[family][dict[char]][col]
    return likelihood

In [355]:
def save_result(input_path, idx, membership):
    output_path = input_path[:input_path.index(".")]
    f = open("%s_%d.txt" % (output_path, idx), 'w')
    for seq in membership:
        f.write("%d %.2f\n" % (np.argmax(seq), seq[np.argmax(seq)]))

In [356]:
input_path = "subseqs11.txt"
seqlen = 10
numfamily = 5
iters = 100
savefreq = 10

In [357]:
# build data array for whole sequence
data = create_data(input_path, seqlen)
seqlen = data.shape[1]
data

created data with size (312, 10)


array([['E', 'K', 'K', ..., 'L', 'G', 'E'],
       ['E', 'K', 'K', ..., 'L', 'G', 'E'],
       ['E', 'G', 'C', ..., 'G', 'S', 'A'],
       ..., 
       ['P', 'I', 'F', ..., 'R', 'L', 'G'],
       ['G', 'I', 'F', ..., 'E', 'N', 'G'],
       ['G', 'I', 'F', ..., 'E', 'N', 'G']], 
      dtype='|S1')

In [358]:
# build dictionary for amino acid from input data
dict = create_dictionary(data)
dict

created dictionary with 22 characters


{'-': 16,
 'A': 10,
 'C': 6,
 'D': 12,
 'E': 0,
 'F': 8,
 'G': 2,
 'H': 3,
 'I': 5,
 'K': 1,
 'L': 4,
 'M': 17,
 'N': 18,
 'P': 15,
 'Q': 19,
 'R': 11,
 'S': 9,
 'T': 13,
 'V': 7,
 'W': 20,
 'X': 21,
 'Y': 14}

In [359]:
# initialize membership array for entire sequences: seqsize x numfamily
membership = create_membership(len(data), numfamily)
membership[:, 0]

created membership matrix of size (312, 5)


array([ 0.02216724,  0.04895927,  0.02073381,  0.13232716,  0.12538002,
        0.02351304,  0.16491289,  0.1952264 ,  0.24454695,  0.28752644,
        0.12227778,  0.2302057 ,  0.13663157,  0.08490586,  0.2090415 ,
        0.23914938,  0.19242978,  0.19210758,  0.46974275,  0.2413094 ,
        0.07096122,  0.31004224,  0.21158041,  0.16076373,  0.19374209,
        0.14028   ,  0.21513021,  0.08302746,  0.30833655,  0.12994471,
        0.2697181 ,  0.16879741,  0.26556068,  0.23608297,  0.18060954,
        0.17437202,  0.34459932,  0.22684193,  0.33973416,  0.25276794,
        0.37487403,  0.27003457,  0.20006754,  0.15968976,  0.11294869,
        0.27942811,  0.28923626,  0.15453418,  0.03386331,  0.36343357,
        0.11702849,  0.45695093,  0.23450769,  0.350846  ,  0.08634306,
        0.23789223,  0.03967877,  0.25803662,  0.37314665,  0.22421044,
        0.21502526,  0.37389134,  0.25992727,  0.34828357,  0.17620215,
        0.10411167,  0.14271734,  0.03687881,  0.08503055,  0.10

In [360]:
# initialize probability matrix for each family: numfamily x len(dict) x seqlen
prob_mat = create_prob(numfamily, len(dict), seqlen)

created probability matrix of size (5, 22, 10)


In [361]:
print("Running EM algorithm..")
for idx in xrange(iters):
    if idx % 10 == 0:
        print("  %dth iter.." % idx)
    if idx % savefreq == 0:
        print("  saving results on %dth iter.." % idx)
        save_result(input_path, idx, membership)
        
    # Expectation
    for family in xrange(1):
        for col in xrange(seqlen):
            for char in dict:
                mask = data[:, col] != char
                # if the $col of data contains no $char,
                if np.ma.all(mask):
                    prob_mat[family][dict[char]][col] = 0
                # if the $col of data contains at least one $char,
                else:    
                    masked_array = np.ma.masked_array(membership[:, family], mask)
                    np.ma.set_fill_value(masked_array, 0)
                    prob_mat[family][dict[char]][col] = masked_array.sum() / sum(membership[:, family])
                
    # Maximization
    for seqidx, seq in enumerate(data):
        for family in xrange(numfamily):
            membership[seqidx, family] = compute_membership(seq, family, dict, prob_mat)
        membership[seqidx] = np.apply_along_axis(normalize, axis=0, arr=membership[seqidx])
            
        
print("DONE..")

Running EM algorithm..
  0th iter..
  saving results on 0th iter..
  10th iter..
  saving results on 10th iter..
  20th iter..
  saving results on 20th iter..
  30th iter..
  saving results on 30th iter..
  40th iter..
  saving results on 40th iter..
  50th iter..
  saving results on 50th iter..
  60th iter..
  saving results on 60th iter..
  70th iter..
  saving results on 70th iter..
  80th iter..
  saving results on 80th iter..
  90th iter..
  saving results on 90th iter..
DONE..
